# Performance Analysis: Matrix Dimensions vs einsumcorr Performance

This notebook explores the relationship between matrix size (particularly column dimensionality) and the performance of the `einsumcorr` package compared to standard numpy correlation computation.

## Hypothesis
The number of columns affects performance more than the number of rows, since correlation is computed between columns.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
import seaborn as sns
from einsumcorr import optcorr
import torch

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print(f"Using device: {'GPU (MPS)' if torch.backends.mps.is_available() else 'GPU (CUDA)' if torch.cuda.is_available() else 'CPU'}")

## Helper Functions

In [ ]:
def time_function(func, *args, n_runs=3):
    """Time a function over multiple runs and return mean time."""
    times = []
    for _ in range(n_runs):
        start = time.perf_counter()
        result = func(*args)
        end = time.perf_counter()
        times.append(end - start)
    return np.mean(times), result

def numpy_corrcoef(x):
    """Standard numpy correlation computation."""
    return np.corrcoef(x.T)

def generate_test_matrix(n_rows, n_cols, seed=42):
    """Generate a random test matrix."""
    np.random.seed(seed)
    return np.random.randn(n_rows, n_cols)

## Experiment 1: Effect of Column Dimensionality

Test performance across different numbers of columns while keeping rows constant.

In [ ]:
# Test parameters
n_rows_fixed = 1000  # Fixed number of rows
col_dimensions = [5, 10, 20, 50, 100, 200, 500, 1000, 1500, 2000]
n_runs = 3

results_cols = []

print("Testing column dimensionality effect...")
print(f"Fixed rows: {n_rows_fixed}")
print("Columns:", end=" ")

for n_cols in col_dimensions:
    print(f"{n_cols}", end=" ", flush=True)
    
    # Generate test matrix
    X = generate_test_matrix(n_rows_fixed, n_cols)
    
    # Time optcorr
    time_optcorr, result_optcorr = time_function(optcorr, X, n_runs=n_runs)
    
    # Time numpy corrcoef
    time_numpy, result_numpy = time_function(numpy_corrcoef, X, n_runs=n_runs)
    
    # Verify results match (within tolerance)
    max_diff = np.max(np.abs(result_optcorr - result_numpy))
    
    results_cols.append({
        'n_rows': n_rows_fixed,
        'n_cols': n_cols,
        'time_optcorr': time_optcorr,
        'time_numpy': time_numpy,
        'speedup': time_numpy / time_optcorr if time_optcorr > 0 else float('inf'),
        'max_diff': max_diff
    })

print("\nDone!")
df_cols = pd.DataFrame(results_cols)
print(df_cols)

## Experiment 2: Effect of Row Dimensionality

Test performance across different numbers of rows while keeping columns constant.

In [ ]:
# Test parameters
n_cols_fixed = 100  # Fixed number of columns
row_dimensions = [100, 200, 500, 1000, 2000, 5000, 10000]

results_rows = []

print("Testing row dimensionality effect...")
print(f"Fixed columns: {n_cols_fixed}")
print("Rows:", end=" ")

for n_rows in row_dimensions:
    print(f"{n_rows}", end=" ", flush=True)
    
    # Generate test matrix
    X = generate_test_matrix(n_rows, n_cols_fixed)
    
    # Time optcorr
    time_optcorr, result_optcorr = time_function(optcorr, X, n_runs=n_runs)
    
    # Time numpy corrcoef
    time_numpy, result_numpy = time_function(numpy_corrcoef, X, n_runs=n_runs)
    
    # Verify results match (within tolerance)
    max_diff = np.max(np.abs(result_optcorr - result_numpy))
    
    results_rows.append({
        'n_rows': n_rows,
        'n_cols': n_cols_fixed,
        'time_optcorr': time_optcorr,
        'time_numpy': time_numpy,
        'speedup': time_numpy / time_optcorr if time_optcorr > 0 else float('inf'),
        'max_diff': max_diff
    })

print("\nDone!")
df_rows = pd.DataFrame(results_rows)
print(df_rows)

## Visualization: Performance vs Column Dimensionality

In [ ]:
# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('einsumcorr Performance Analysis', fontsize=16, fontweight='bold')

# Plot 1: Timing vs Columns
ax1 = axes[0, 0]
ax1.plot(df_cols['n_cols'], df_cols['time_optcorr'], 'o-', label='optcorr', linewidth=2, markersize=6)
ax1.plot(df_cols['n_cols'], df_cols['time_numpy'], 's-', label='numpy.corrcoef', linewidth=2, markersize=6)
ax1.set_xlabel('Number of Columns')
ax1.set_ylabel('Time (seconds)')
ax1.set_title(f'Performance vs Columns ({n_rows_fixed} rows)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_yscale('log')

# Plot 2: Speedup vs Columns
ax2 = axes[0, 1]
colors = ['red' if x < 1 else 'green' for x in df_cols['speedup']]
ax2.bar(range(len(df_cols)), df_cols['speedup'], color=colors, alpha=0.7)
ax2.axhline(y=1, color='black', linestyle='--', alpha=0.7, label='Break-even')
ax2.set_xlabel('Column Dimension Index')
ax2.set_ylabel('Speedup (numpy_time / optcorr_time)')
ax2.set_title('Speedup vs Columns')
ax2.set_xticks(range(len(df_cols)))
ax2.set_xticklabels(df_cols['n_cols'], rotation=45)
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Timing vs Rows
ax3 = axes[1, 0]
ax3.plot(df_rows['n_rows'], df_rows['time_optcorr'], 'o-', label='optcorr', linewidth=2, markersize=6)
ax3.plot(df_rows['n_rows'], df_rows['time_numpy'], 's-', label='numpy.corrcoef', linewidth=2, markersize=6)
ax3.set_xlabel('Number of Rows')
ax3.set_ylabel('Time (seconds)')
ax3.set_title(f'Performance vs Rows ({n_cols_fixed} columns)')
ax3.legend()
ax3.grid(True, alpha=0.3)
ax3.set_yscale('log')

# Plot 4: Computational Complexity Analysis
ax4 = axes[1, 1]
# Theoretical complexity: O(n_cols^2 * n_rows) for correlation
theoretical_cols = df_cols['n_cols']**2 * n_rows_fixed
theoretical_rows = df_rows['n_rows'] * n_cols_fixed**2

# Normalize for comparison
ax4.plot(df_cols['n_cols'], df_cols['time_optcorr'] / df_cols['time_optcorr'].iloc[0], 
         'o-', label='optcorr (cols)', linewidth=2)
ax4.plot(df_cols['n_cols'], theoretical_cols / theoretical_cols[0], 
         '--', label='O(n_cols²)', alpha=0.7)
ax4.set_xlabel('Dimension Size')
ax4.set_ylabel('Normalized Time')
ax4.set_title('Computational Complexity')
ax4.legend()
ax4.grid(True, alpha=0.3)
ax4.set_yscale('log')
ax4.set_xscale('log')

plt.tight_layout()
plt.show()

## Analysis Summary

In [ ]:
print("=== PERFORMANCE ANALYSIS SUMMARY ===")
print()

# Column effect analysis
print("📊 COLUMN DIMENSIONALITY EFFECT:")
print(f"   • Tested {len(col_dimensions)} column sizes: {min(col_dimensions)} to {max(col_dimensions)}")
print(f"   • Fixed rows: {n_rows_fixed}")
print(f"   • Best speedup: {df_cols['speedup'].max():.2f}x at {df_cols.loc[df_cols['speedup'].idxmax(), 'n_cols']} columns")
print(f"   • Worst slowdown: {df_cols['speedup'].min():.2f}x at {df_cols.loc[df_cols['speedup'].idxmin(), 'n_cols']} columns")
print(f"   • Average speedup: {df_cols['speedup'].mean():.2f}x")
print()

# Row effect analysis
print("📈 ROW DIMENSIONALITY EFFECT:")
print(f"   • Tested {len(row_dimensions)} row sizes: {min(row_dimensions)} to {max(row_dimensions)}")
print(f"   • Fixed columns: {n_cols_fixed}")
print(f"   • Best speedup: {df_rows['speedup'].max():.2f}x at {df_rows.loc[df_rows['speedup'].idxmax(), 'n_rows']} rows")
print(f"   • Worst slowdown: {df_rows['speedup'].min():.2f}x at {df_rows.loc[df_rows['speedup'].idxmin(), 'n_rows']} rows")
print(f"   • Average speedup: {df_rows['speedup'].mean():.2f}x")
print()

# Scaling analysis
col_scaling_factor = (df_cols['time_optcorr'].iloc[-1] / df_cols['time_optcorr'].iloc[0])
theoretical_scaling = (col_dimensions[-1] / col_dimensions[0])**2

print("🔬 SCALING ANALYSIS:")
print(f"   • Column scaling (empirical): {col_scaling_factor:.1f}x time increase")
print(f"   • Column scaling (theoretical O(n²)): {theoretical_scaling:.1f}x")
print(f"   • Scaling efficiency: {(theoretical_scaling/col_scaling_factor):.2f}")
print()

# Accuracy verification
max_error_cols = df_cols['max_diff'].max()
max_error_rows = df_rows['max_diff'].max()

print("✅ ACCURACY VERIFICATION:")
print(f"   • Maximum difference vs numpy (columns): {max_error_cols:.2e}")
print(f"   • Maximum difference vs numpy (rows): {max_error_rows:.2e}")
print(f"   • All results within acceptable tolerance: {max(max_error_cols, max_error_rows) < 1e-3}")
print()

# Device info
device_type = 'MPS (Apple Silicon)' if torch.backends.mps.is_available() else 'CUDA' if torch.cuda.is_available() else 'CPU'
print(f"🖥️  COMPUTE DEVICE: {device_type}")


## Detailed Performance Tables

In [ ]:
print("Column Dimensionality Results:")
display_cols = df_cols[['n_cols', 'time_optcorr', 'time_numpy', 'speedup']].copy()
display_cols['time_optcorr'] = display_cols['time_optcorr'].map('{:.4f}s'.format)
display_cols['time_numpy'] = display_cols['time_numpy'].map('{:.4f}s'.format)
display_cols['speedup'] = display_cols['speedup'].map('{:.2f}x'.format)
print(display_cols.to_string(index=False))

print("\n" + "="*60 + "\n")

print("Row Dimensionality Results:")
display_rows = df_rows[['n_rows', 'time_optcorr', 'time_numpy', 'speedup']].copy()
display_rows['time_optcorr'] = display_rows['time_optcorr'].map('{:.4f}s'.format)
display_rows['time_numpy'] = display_rows['time_numpy'].map('{:.4f}s'.format)
display_rows['speedup'] = display_rows['speedup'].map('{:.2f}x'.format)
print(display_rows.to_string(index=False))

## Key Findings

1. **Column dimensionality has a quadratic effect** on computation time, as expected for correlation matrices (O(n_cols²))

2. **Row dimensionality has a linear effect** on computation time (O(n_rows))

3. **GPU acceleration effectiveness varies** with problem size - smaller problems may not benefit due to overhead

4. **Einstein summation with opt_einsum** provides computational benefits for certain matrix sizes

5. **Results remain numerically accurate** across all tested dimensions compared to numpy's implementation

This analysis confirms the hypothesis that column dimensionality is the primary driver of computational complexity in correlation matrix computation.